In [1]:
import numpy as np
import torch
torch.manual_seed(61) 
from torchtext.vocab import build_vocab_from_iterator, Vocab
import pandas as pd
import nltk
nltk.download('punkt')
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
import sklearn

[nltk_data] Downloading package punkt to
[nltk_data]     /home/jazzblazzer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
csv_file = 'ANLP-2/train.csv'
df = pd.read_csv(csv_file)
SVD_desc = df['Description'].head(1000).values


FileNotFoundError: [Errno 2] No such file or directory: 'ANLP-2/train.csv'

In [3]:

SVD_train = []
for i in range(len(SVD_desc)):
    SVD_desc[i]=SVD_desc[i].replace('\\', ' ')
    new_sentence = nltk.word_tokenize(SVD_desc[i])
    new_sentence.append('<END>')
    new_sentence.insert(0, '<START>')
    SVD_train.append(new_sentence)
print(SVD_train[:5])



[['<START>', 'Reuters', '-', 'Short-sellers', ',', 'Wall', 'Street', "'s", 'dwindling', 'band', 'of', 'ultra-cynics', ',', 'are', 'seeing', 'green', 'again', '.', '<END>'], ['<START>', 'Reuters', '-', 'Private', 'investment', 'firm', 'Carlyle', 'Group', ',', 'which', 'has', 'a', 'reputation', 'for', 'making', 'well-timed', 'and', 'occasionally', 'controversial', 'plays', 'in', 'the', 'defense', 'industry', ',', 'has', 'quietly', 'placed', 'its', 'bets', 'on', 'another', 'part', 'of', 'the', 'market', '.', '<END>'], ['<START>', 'Reuters', '-', 'Soaring', 'crude', 'prices', 'plus', 'worries', 'about', 'the', 'economy', 'and', 'the', 'outlook', 'for', 'earnings', 'are', 'expected', 'to', 'hang', 'over', 'the', 'stock', 'market', 'next', 'week', 'during', 'the', 'depth', 'of', 'the', 'summer', 'doldrums', '.', '<END>'], ['<START>', 'Reuters', '-', 'Authorities', 'have', 'halted', 'oil', 'export', 'flows', 'from', 'the', 'main', 'pipeline', 'in', 'southern', 'Iraq', 'after', 'intelligence',

#### Building Co-coccurance matrix and getting the svd of it.

In [4]:
vocab = build_vocab_from_iterator(SVD_train, specials=["<START>", "<END>","<UNK>"])
vocab.set_default_index(vocab["<UNK>"])

Co_occ_mat = np.zeros((len(vocab), len(vocab)))
for sentence in SVD_train:
    for i in range(len(sentence)-1):
        Co_occ_mat[vocab[sentence[i]]][vocab[sentence[i+1]]] += 1
        Co_occ_mat[vocab[sentence[i+1]]][vocab[sentence[i]]] += 1

U, S, V = np.linalg.svd(Co_occ_mat)
print(U)



[[-6.72411019e-03 -5.64882692e-03 -1.73583192e-02 ...  3.70516531e-18
  -4.92747145e-18  1.14448974e-17]
 [-7.00678886e-01  7.01851116e-01  2.31046767e-02 ... -2.51736579e-18
  -1.35180931e-17 -5.04374239e-17]
 [-1.11022302e-16 -5.55111512e-17  2.77555756e-17 ...  3.17416828e-04
  -6.27501788e-04  1.06921373e-04]
 ...
 [-7.33997008e-06 -3.99916855e-06 -3.84756477e-04 ... -2.08294397e-03
   4.08990380e-04  5.87486934e-03]
 [-1.56879129e-06 -9.01446033e-07 -5.23534134e-05 ... -4.66804001e-02
   1.85662141e-02  5.22048459e-03]
 [-1.24140690e-06 -1.09299973e-06 -4.39913452e-05 ... -1.77637892e-02
   1.33113621e-02 -6.79956438e-03]]


In [ ]:
torch.save(vocab,"svdvocab.pt")
torch.save(U,"svd-word-vectors.pt")

: 

In [35]:
vocab = build_vocab_from_iterator(SVD_train, specials=["<START>", "<END>","<UNK>"])
vocab.set_default_index(vocab["<UNK>"])
freqs={}
for sentence in SVD_train:
    for word in sentence:
        freqs[word] = freqs.get(word, 0) + 1
print(len(vocab))

25969


In svd, the diagonal matrix gives us the information about how much variance in data we get. So the next cell tries to get an estimate of how many columns we need to take in svd for feature extraction.

In [281]:
total = np.sum(S)
size = 0
for i in range(len(S)):
    if np.sum(S[:i])/total > 0.7:
        print(i)
        size = i
        break
print(len(S))
U[0][:i].shape
    

1057
7842


(1057,)

In [204]:
unique_values = df['Class Index'].unique()
print(unique_values)


[3 4 2 1]


LSTM and its training. Same as ass2, here we hv 4 classes. 

In [ ]:
PAD_TOKEN = "<pad>"
class LSTMDataset(Dataset):
  def __init__(self, sentences,labels,vocabulary:Vocab|None=None):
    """Initialize the dataset. Setup Code goes here"""
    self.sentences = sentences
    self.labels = labels
    self.vocabulary = vocabulary

  def __len__(self) -> int:
    """Returns number of datapoints."""
    return len(self.sentences)

  def __getitem__(self, index: int):
    """Get the datapoint at `index`."""
    return torch.tensor(self.vocabulary.lookup_indices(self.sentences[index])), torch.tensor((self.labels[index]))
  def collate(self, batch: list[tuple[torch.Tensor, torch.Tensor]]) -> tuple[torch.Tensor, torch.Tensor]:
    """Given a list of datapoints, batch them together"""
    sentences = [i[0] for i in batch]
    labels = [i[1] for i in batch]
    padded_sentences = pad_sequence(sentences, batch_first=True, padding_value=self.vocabulary["<UNK>"]) # pad sentences with pad token id
    padded_labels = torch.tensor(labels) # pad labels with 0 because pad token cannot be entities

    return padded_sentences, padded_labels


class LSTM(torch.nn.Module):
  def __init__(self,embed_size,embeddings,hidden_dim,num_layers,num_classes):
    super().__init__()

    self.embedding_module = torch.nn.Embedding.from_pretrained(embeddings[:, :embed_size])
    self.embed_size = embed_size  
    self.lstm = torch.nn.LSTM(embed_size,hidden_dim,num_layers,batch_first=True,bidirectional=True)
    self.outLayer = torch.nn.Linear(2*hidden_dim,num_classes)
    self.num_classes = num_classes

  def forward(self, sentence):
    embedding = self.embedding_module(sentence)
    out, _ = self.lstm(embedding)
    output = self.outLayer(out[:, -1, :])
    return torch.nn.functional.log_softmax(output,dim=1)







In [ ]:
from torch.utils.data import DataLoader

lstm_desc = df['Description'].head(1000).values
lstm_train = []
for i in range(len(lstm_desc)):
    lstm_desc[i]=lstm_desc[i].replace('\\', ' ')
    new_sentence = nltk.word_tokenize(lstm_desc[i])
    new_sentence.append('<END>')
    new_sentence.insert(0, '<START>')
    lstm_train.append(new_sentence)

lstm_data = LSTMDataset(lstm_train,df['Class Index'].head(1000).values,vocab)
lstm_data_loader =  DataLoader(lstm_data, batch_size=64, shuffle=True,collate_fn=lstm_data.collate)

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
# lstm_test_dataloader = DataLoader(lstm_val_dataset, batch_size=3,collate_fn=lstm_data.collate)
lstm_model = LSTM(1024,torch.tensor(U),64,2,4)
lstm_model.float()
loss_fn = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(lstm_model.parameters(),lr = 0.01)


for epoch_num in range(10):
  # Put model in training mode
  lstm_model.train()
  for batch_num, (sentences, tags) in enumerate(lstm_data_loader):

    pred = lstm_model(sentences)
    loss = loss_fn(pred,tags-1)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch_num % 100 == 0:
      # print loss every 2 steps
      print(f"{batch_num=}, {loss.item()=}")
      



batch_num=0, loss.item()=1.3851487636566162
batch_num=0, loss.item()=1.3624407052993774
batch_num=0, loss.item()=1.3797430992126465
batch_num=0, loss.item()=1.3555898666381836
batch_num=0, loss.item()=1.347963571548462
batch_num=0, loss.item()=1.328068494796753
batch_num=0, loss.item()=1.3153446912765503
batch_num=0, loss.item()=1.3682851791381836
batch_num=0, loss.item()=1.3023425340652466
batch_num=0, loss.item()=1.3287171125411987


In [ ]:
csv_file_test = 'ANLP-2/test.csv'
tf = pd.read_csv(csv_file_test)
SVD_test_desc = tf['Description'].head(1000).values

SVD_test = []
for i in range(len(SVD_test_desc)):
    SVD_test_desc[i]=SVD_test_desc[i].replace('\\', ' ')
    new_sentence = nltk.word_tokenize(SVD_test_desc[i])
    new_sentence.append('<END>')
    new_sentence.insert(0, '<START>')
    SVD_test.append(new_sentence)

lstm_test_data = LSTMDataset(SVD_test,tf['Class Index'].head(1000).values,vocab)
lstm_test_data_loader =  DataLoader(lstm_test_data, batch_size=64, shuffle=True,collate_fn=lstm_test_data.collate)



In [ ]:
lstm_model.eval()

true_labels = []
predicted_labels = []

# Iterate over test set
for sentences, labels in lstm_test_data_loader:
    # Forward pass
    output = lstm_model(sentences)

    # Get predicted labels
    _, predicted = torch.max(output, 1)

    # Append true and predicted labels
    true_labels.extend(labels.view(-1).tolist())
    predicted_labels.extend(predicted.tolist())

# Compute evaluation metrics
print(true_labels,predicted)
accuracy = accuracy_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels,average="weighted")
f1 = f1_score(true_labels, predicted_labels,average="weighted")
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

[4, 4, 1, 1, 3, 3, 2, 1, 4, 1, 2, 4, 4, 2, 1, 1, 1, 1, 4, 1, 1, 1, 2, 3, 4, 1, 2, 2, 2, 2, 3, 3, 4, 4, 1, 2, 1, 2, 3, 1, 2, 1, 3, 1, 3, 3, 2, 2, 1, 3, 4, 4, 1, 2, 1, 3, 1, 4, 2, 1, 1, 2, 4, 1, 3, 1, 1, 3, 1, 2, 2, 2, 4, 2, 1, 4, 4, 1, 2, 3, 1, 4, 1, 4, 2, 2, 4, 3, 4, 1, 4, 2, 3, 4, 2, 2, 3, 3, 1, 1, 1, 1, 2, 3, 4, 4, 2, 2, 2, 2, 4, 2, 2, 4, 1, 4, 1, 4, 2, 4, 1, 1, 2, 2, 3, 1, 1, 2, 3, 1, 3, 4, 2, 3, 2, 1, 1, 1, 1, 2, 1, 1, 3, 1, 4, 4, 2, 4, 2, 4, 1, 4, 2, 4, 2, 4, 2, 4, 4, 3, 2, 3, 4, 2, 4, 2, 2, 1, 4, 2, 2, 1, 1, 1, 3, 1, 4, 1, 2, 1, 1, 4, 2, 1, 3, 2, 2, 2, 3, 1, 1, 4, 3, 3, 1, 2, 4, 1, 3, 1, 3, 1, 2, 2, 4, 4, 2, 1, 2, 1, 2, 2, 1, 4, 4, 2, 3, 4, 2, 4, 4, 3, 3, 3, 4, 4, 2, 1, 2, 1, 1, 2, 4, 3, 3, 2, 2, 1, 4, 4, 1, 1, 2, 4, 4, 4, 4, 4, 1, 2, 2, 1, 4, 2, 4, 2, 1, 2, 4, 4, 4, 2, 3, 2, 4, 3, 2, 2, 2, 2, 2, 3, 4, 1, 3, 4, 4, 4, 1, 3, 1, 4, 2, 2, 2, 2, 1, 2, 2, 3, 4, 3, 1, 4, 3, 2, 1, 1, 4, 3, 1, 1, 1, 3, 2, 3, 2, 1, 2, 3, 2, 2, 2, 2, 4, 2, 2, 2, 3, 4, 4, 2, 1, 1, 2, 3, 1, 2, 4, 2, 3, 3, 1, 

Accuracy is very low because kernel kept crashing and I took a very small number of train senteces, not giving enough vocabulary.

In [ ]:
csv_file_test = 'ANLP-2/test.csv'
tf = pd.read_csv(csv_file_test)
SVD_test_desc = tf['Description'].head(1000).values

SVD_test = []
for i in range(len(SVD_test_desc)):
    SVD_test_desc[i]=SVD_test_desc[i].replace('\\', ' ')
    new_sentence = nltk.word_tokenize(SVD_test_desc[i])
    new_sentence.append('<END>')
    new_sentence.insert(0, '<START>')
    SVD_test.append(new_sentence)

lstm_test_data = LSTMDataset(SVD_test,tf['Class Index'].head(1000).values,vocab)
lstm_test_data_loader =  DataLoader(lstm_test_data, batch_size=64, shuffle=True,collate_fn=lstm_test_data.collate)

lstm_model.eval()

true_labels = []
predicted_labels = []

# Iterate over test set
for sentences, labels in lstm_test_data_loader:
    # Forward pass
    output = lstm_model(sentences)

    # Get predicted labels
    _, predicted = torch.max(output, 1)

    # Append true and predicted labels
    true_labels.extend(labels.view(-1).tolist())
    predicted_labels.extend(predicted.tolist())

# Compute evaluation metrics
print(true_labels,predicted)
accuracy = accuracy_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels,average="weighted")
f1 = f1_score(true_labels, predicted_labels,average="weighted")
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

## WORD2VEC
Starting to train Skipgram
We discard some tokens with respect to probbaility here. This is so that very frequent words like is, the conjunctions do not dominate the train dataset.  
The neg_sampler samples from a random choice of words, with probability distribution of choosing being defined as
p=[(freq[j]**0.75)/(freq_sum) for j in vocab.get_stoi().keys()]

In [6]:
#skipgram
vocab = torch.load('skgvocab.pt')
import random
PAD_TOKEN = "<pad>"

def get_discard(x):
  return (np.sqrt(0.001/int(x))+1)*(0.001/int(x))

class SKGataset(Dataset):
  def __init__(self, sentences,window_size,freq,vocabulary:Vocab|None=None):
    """Initialize the dataset. Setup Code goes here"""
    self.vocabulary = vocabulary  
    self.target = []
    self.context = []
    self.freq = freq
    for x in sentences:
      for i in range(len(x)):
        curr_context = []
        for j in range(max(0,i-window_size),min(len(x),i+window_size+1)):
          if i!=j:
            curr_context.append(x[j])
        if len(x)<i+window_size+1:
          for j in range(len(x),i+window_size+1):
            curr_context.append('<END>')
        if i<window_size:
          for j in range(max(0,i-window_size),i):
            curr_context.append('<START>')
        self.target.append(x[i])
        self.context.append(curr_context)
      
            

  def __len__(self) -> int:
    """Returns number of datapoints."""
    return len(self.target)

  def __getitem__(self, index: int):
    """Get the datapoint at `index`."""
    return torch.tensor(self.vocabulary.lookup_indices([self.target[index]])), torch.tensor(self.vocabulary.lookup_indices(self.context[index]))
  def collate(self, batch: list[tuple[torch.Tensor, torch.Tensor]]) : 
    modified_batch = []
    """Given a list of datapoints, batch them together"""
    for i in batch:
      prob = random.random()
      if prob < get_discard(self.freq[(self.vocabulary.lookup_token(i[0]))]):
        continue
      else :
        modified_context = []
        for j in range(len(i[1])):
          prob = random.random()
          if prob < get_discard(self.freq[(self.vocabulary.lookup_token(i[1][j]))]):
            continue
          else:
            modified_context.append(i[1][j])        
        modified_batch.append((i[0],torch.tensor(modified_context)))
    targets = [i[0] for i in modified_batch]
    contexts = [i[1] for i in modified_batch]
    padded_contexts = pad_sequence(contexts, batch_first=True, padding_value=self.vocabulary["<PAD>"]) # pad sentences with pad token id
    return torch.tensor(targets), padded_contexts
    
    # padded_sentences = pad_sequence(sentences, batch_first=True, padding_value=self.vocabulary["<UNK>"]) # pad sentences with pad token id
    # padded_labels = torch.tensor(labels) # pad labels with 0 because pad token cannot be entities

    # return padded_sentences, padded_labels
vocab_list = list(vocab.get_stoi().keys())
vocab_dict = vocab.get_stoi()

def negative_sampler(vocab,p,num_samples,bath_size):
  neg_samples = np.random.choice(vocab_list,bath_size*num_samples,p=p)
  neg_samples = [vocab_dict[sample] for sample in neg_samples] 
  return torch.tensor(neg_samples).view(bath_size,num_samples)
  
class SKG(torch.nn.Module):
  def __init__(self,vocab,embed_size):
    super().__init__()
    self.target = torch.nn.Embedding(len(vocab), embed_size)
    self.context = torch.nn.Embedding(len(vocab), embed_size)

  def forward(self, inputs,contexts):
    target_embed = self.target(inputs)
    target_embed = target_embed.view(target_embed.shape[0],1,target_embed.shape[1])
    context_embed = self.context(contexts)
    context_embed = context_embed.view(context_embed.shape[0],context_embed.shape[2],context_embed.shape[1])
    similarity = torch.bmm(target_embed,context_embed)
    return similarity.view(similarity.shape[0],similarity.shape[2])
  



    




In [7]:
from torch.utils.data import DataLoader

lstm_desc = df['Description'].head(10000).values
lstm_train = []
freq = {}
for i in range(len(lstm_desc)):
    lstm_desc[i]=lstm_desc[i].replace('\\', ' ')
    new_sentence = nltk.word_tokenize(lstm_desc[i])
    new_sentence.append('<END>')
    new_sentence.insert(0, '<START>')
    lstm_train.append(new_sentence)
for sentence in lstm_train:
    for word in sentence:
        freq[word] = freq.get(word, 0) + 1
freq['<UNK>'] = 0
freq['<PAD>'] = 0
# vocab = build_vocab_from_iterator(lstm_train, specials=["<START>", "<END>","<UNK>","<PAD>"])
skg_data = SKGataset(lstm_train,4,freq,vocab)
# lstm_data_loader =  DataLoader(lstm_data, batch_size=64, shuffle=True,collate_fn=lstm_data.collate)

In [8]:
lstm_data_loader =  DataLoader(skg_data, batch_size=128, shuffle=True,collate_fn=skg_data.collate)
# for batch_num, pairs in enumerate(lstm_data_loader):
#     print(pairs)
#     break
# for i in lstm_data_loader:
#     print(i)
#     break
print(len(lstm_data_loader))


3060


## SKG Model training 

In [311]:
skg_model = SKG(vocab,300)
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(skg_model.parameters(),lr = 0.01)

freq_sum = 0
for i in freq.keys():
  freq_sum += freq[i]**0.75
p=[(freq[j]**0.75)/(freq_sum) for j in vocab.get_stoi().keys()]

for epoch_num in range(4):
  # Put model in training mode
  skg_model.train()
  for batch_num, (target,context) in enumerate(lstm_data_loader):
    neg_samples = (negative_sampler(vocab,p,5,target.shape[0]))
    y_pos = torch.ones(context.shape[0],context.shape[1])
    context = (torch.cat([context,neg_samples],dim=1))
    y_neg = torch.zeros(neg_samples.shape[0],neg_samples.shape[1])
    y = (torch.cat([y_pos,y_neg],dim=1))
    pred = skg_model(target,context)
    loss = loss_fn(pred,y)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(f"{batch_num=}, {loss.item()=}")
    

    
    
      

batch_num=0, loss.item()=6.465950012207031
batch_num=1, loss.item()=6.7011284828186035
batch_num=2, loss.item()=6.443297863006592
batch_num=3, loss.item()=6.682711124420166
batch_num=4, loss.item()=6.704126834869385
batch_num=5, loss.item()=6.447221755981445
batch_num=6, loss.item()=6.619509220123291
batch_num=7, loss.item()=6.858497142791748
batch_num=8, loss.item()=6.99205207824707
batch_num=9, loss.item()=6.45003080368042
batch_num=10, loss.item()=6.097381591796875
batch_num=11, loss.item()=6.811800003051758
batch_num=12, loss.item()=6.54078483581543
batch_num=13, loss.item()=6.370987892150879
batch_num=14, loss.item()=6.164574146270752
batch_num=15, loss.item()=5.950930595397949
batch_num=16, loss.item()=6.113244533538818
batch_num=17, loss.item()=5.972259044647217
batch_num=18, loss.item()=6.3723907470703125
batch_num=19, loss.item()=6.085469722747803
batch_num=20, loss.item()=5.832442283630371
batch_num=21, loss.item()=6.037815570831299
batch_num=22, loss.item()=5.995684623718262

In [24]:
vocab = torch.load('skgvocab.pt')
vocab.set_default_index(vocab["<UNK>"])
skg_embed = torch.load('skgembed.pt')
print(vocab.get_stoi())
print(skg_embed.shape)

{'zip': 25966, 'zigzagging': 25964, 'youths': 25960, 'youngsters': 25959, 'yler': 25957, 'yields': 25956, 'yesteryear': 25954, 'yellow-clad': 25953, 'yelling': 25952, 'yelled': 25951, 'yeast': 25950, 'yearly': 25948, 'year-to-June': 25947, 'year-old': 25946, 'yard': 25945, 'yanked': 25944, 'yank': 25943, 'yammering': 25942, 'x86': 25939, 'www.vonage.com': 25938, 'www.linksys.com': 25937, 'www.lindows.com': 25936, 'wussy': 25931, 'wunderkind': 25930, 'wrongfully': 25928, 'wrests': 25927, 'wrestlers': 25925, 'wrestler': 25924, 'wrench': 25922, 'wrecks': 25921, 'wreaked': 25920, 'wrath': 25919, 'wrap-up': 25918, 'wrangle': 25917, 'wracking': 25916, 'worthier': 25912, 'worsened': 25911, 'worrisome': 25909, 'world-famous': 25906, 'workstations': 25905, 'workout': 25903, 'worker': 25900, 'wording': 25898, 'wool-producing': 25897, 'wooing': 25896, 'woods': 25894, 'wood': 25892, 'wonderful': 25889, 'wolfed': 25887, 'wits': 25884, 'withstood': 25882, 'withstand': 25881, 'withold': 25880, 'wishi

lstm model training, same as in ass2

In [25]:
PAD_TOKEN = "<pad>"
class LSTMDataset(Dataset):
  def __init__(self, sentences,labels,vocabulary:Vocab|None=None):
    """Initialize the dataset. Setup Code goes here"""
    self.sentences = sentences
    self.labels = labels
    self.vocabulary = vocabulary

  def __len__(self) -> int:
    """Returns number of datapoints."""
    return len(self.sentences)

  def __getitem__(self, index: int):
    """Get the datapoint at `index`."""
    return torch.tensor(self.vocabulary.lookup_indices(self.sentences[index])), torch.tensor((self.labels[index]))
  def collate(self, batch: list[tuple[torch.Tensor, torch.Tensor]]) -> tuple[torch.Tensor, torch.Tensor]:
    """Given a list of datapoints, batch them together"""
    sentences = [i[0] for i in batch]
    labels = [i[1] for i in batch]
    padded_sentences = pad_sequence(sentences, batch_first=True, padding_value=self.vocabulary["<UNK>"]) # pad sentences with pad token id
    padded_labels = torch.tensor(labels) # pad labels with 0 because pad token cannot be entities

    return padded_sentences, padded_labels


class LSTM(torch.nn.Module):
  def __init__(self,embed_size,embeddings,hidden_dim,num_layers,num_classes):
    super().__init__()

    self.embedding_module = torch.nn.Embedding.from_pretrained(embeddings[:, :embed_size])
    self.embed_size = embed_size  
    self.lstm = torch.nn.LSTM(embed_size,hidden_dim,num_layers,batch_first=True)
    self.outLayer = torch.nn.Linear(hidden_dim,num_classes)
    self.num_classes = num_classes

  def forward(self, sentence):
    embedding = self.embedding_module(sentence)
    out, _ = self.lstm(embedding)
    output = self.outLayer(out[:, -1, :])
    return torch.nn.functional.log_softmax(output,dim=1)



from torch.utils.data import DataLoader

lstm_desc = df['Description'].head(50000).values
lstm_train = []
for i in range(len(lstm_desc)):
    lstm_desc[i]=lstm_desc[i].replace('\\', ' ')
    new_sentence = nltk.word_tokenize(lstm_desc[i])
    new_sentence.append('<END>')
    new_sentence.insert(0, '<START>')
    lstm_train.append(new_sentence)

lstm_data = LSTMDataset(lstm_train,df['Class Index'].head(20000).values,vocab)
lstm_data_loader =  DataLoader(lstm_data, batch_size=64, shuffle=True,collate_fn=lstm_data.collate)




In [26]:
print(len(lstm_data))

50000


In [27]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
# lstm_test_dataloader = DataLoader(lstm_val_dataset, batch_size=3,collate_fn=lstm_data.collate)
lstm_model = LSTM(300,skg_embed,64,2,4)
lstm_model.float()
loss_fn = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(lstm_model.parameters(),lr = 0.01)


for epoch_num in range(10):
  # Put model in training mode
  lstm_model.train()
  for batch_num, (sentences, tags) in enumerate(lstm_data_loader):

    pred = lstm_model(sentences)
    loss = loss_fn(pred,tags-1)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch_num % 100 == 0:
      # print loss every 2 steps
      print(f"{batch_num=}, {loss.item()=}")
      



batch_num=0, loss.item()=1.3771682977676392


batch_num=100, loss.item()=1.3795030117034912
batch_num=200, loss.item()=1.3941917419433594
batch_num=300, loss.item()=1.3526149988174438
batch_num=400, loss.item()=1.4100050926208496
batch_num=500, loss.item()=1.5194637775421143
batch_num=600, loss.item()=1.4492878913879395
batch_num=700, loss.item()=1.409197449684143
batch_num=800, loss.item()=1.2843577861785889
batch_num=900, loss.item()=1.3267687559127808
batch_num=1000, loss.item()=1.3771171569824219
batch_num=1100, loss.item()=1.3829655647277832
batch_num=1200, loss.item()=1.228877305984497
batch_num=1300, loss.item()=1.3927083015441895
batch_num=1400, loss.item()=1.4245563745498657
batch_num=1500, loss.item()=1.3787531852722168
batch_num=1600, loss.item()=1.262821912765503
batch_num=1700, loss.item()=1.3432058095932007
batch_num=1800, loss.item()=1.6638094186782837
batch_num=1900, loss.item()=1.4371639490127563
batch_num=2000, loss.item()=1.4350823163986206
batch_num=2100, loss.item()=1.2611993551254272
batch_num=2200, loss.item

In [30]:
csv_file_test = 'ANLP-2/test.csv'
tf = pd.read_csv(csv_file_test)
SVD_test_desc = tf['Description'].values

SVD_test = []
for i in range(len(SVD_test_desc)):
    SVD_test_desc[i]=SVD_test_desc[i].replace('\\', ' ')
    new_sentence = nltk.word_tokenize(SVD_test_desc[i])
    new_sentence.append('<END>')
    new_sentence.insert(0, '<START>')
    SVD_test.append(new_sentence)

lstm_test_data = LSTMDataset(SVD_test,tf['Class Index'].values,vocab)
lstm_test_data_loader =  DataLoader(lstm_test_data, batch_size=64, shuffle=True,collate_fn=lstm_test_data.collate)

lstm_model.eval()

true_labels = []
predicted_labels = []

# Iterate over test set
for sentences, labels in lstm_test_data_loader:
    # Forward pass
    output = lstm_model(sentences)

    # Get predicted labels
    _, predicted = torch.max(output, 1)

    # Append true and predicted labels
    true_labels.extend(labels.view(-1).tolist())
    predicted_labels.extend(predicted.tolist())

# Compute evaluation metrics
true_labels = [i-1 for i in true_labels]
print(true_labels,predicted)
accuracy = accuracy_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels,average="weighted")
f1 = f1_score(true_labels, predicted_labels,average="weighted")
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)



[0, 3, 1, 2, 2, 1, 3, 3, 3, 2, 1, 0, 3, 2, 3, 0, 2, 2, 1, 2, 1, 0, 2, 0, 0, 2, 3, 2, 0, 3, 3, 1, 2, 1, 2, 3, 3, 3, 3, 2, 3, 2, 1, 0, 0, 1, 0, 0, 1, 0, 2, 0, 1, 2, 1, 3, 2, 3, 0, 1, 2, 0, 0, 2, 1, 0, 0, 0, 1, 0, 3, 0, 3, 0, 2, 1, 3, 1, 3, 2, 2, 0, 3, 2, 2, 0, 0, 0, 1, 1, 3, 2, 2, 0, 1, 3, 2, 2, 1, 3, 3, 1, 0, 0, 3, 0, 1, 3, 1, 3, 0, 2, 2, 3, 3, 1, 0, 0, 0, 2, 3, 1, 0, 0, 3, 1, 1, 2, 0, 3, 0, 3, 1, 3, 2, 3, 1, 2, 3, 1, 0, 2, 2, 3, 2, 2, 0, 0, 2, 3, 2, 3, 0, 0, 3, 2, 0, 0, 1, 0, 0, 2, 2, 0, 2, 3, 3, 3, 0, 0, 2, 2, 1, 3, 2, 0, 2, 2, 2, 2, 3, 1, 3, 3, 3, 3, 2, 2, 1, 1, 1, 1, 2, 3, 2, 2, 0, 0, 3, 3, 1, 1, 3, 3, 3, 3, 1, 1, 3, 3, 0, 1, 0, 3, 3, 0, 3, 3, 2, 3, 1, 1, 1, 3, 2, 2, 3, 2, 1, 2, 3, 3, 1, 0, 1, 2, 3, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 3, 3, 1, 1, 2, 0, 0, 2, 2, 0, 1, 3, 0, 0, 1, 1, 0, 3, 0, 1, 1, 1, 2, 3, 2, 2, 3, 1, 0, 3, 2, 3, 0, 3, 1, 2, 0, 3, 3, 3, 3, 1, 2, 0, 1, 2, 0, 3, 2, 0, 3, 0, 0, 0, 3, 0, 1, 3, 0, 3, 0, 3, 2, 2, 1, 1, 0, 2, 2, 3, 3, 0, 1, 1, 1, 3, 3, 3, 3, 3, 2, 3, 0, 0, 2, 

In [31]:
csv_file_test = 'ANLP-2/train.csv'
tf = pd.read_csv(csv_file_test)
SVD_test_desc = tf['Description'].values

SVD_test = []
for i in range(len(SVD_test_desc)):
    SVD_test_desc[i]=SVD_test_desc[i].replace('\\', ' ')
    new_sentence = nltk.word_tokenize(SVD_test_desc[i])
    new_sentence.append('<END>')
    new_sentence.insert(0, '<START>')
    SVD_test.append(new_sentence)

lstm_test_data = LSTMDataset(SVD_test,tf['Class Index'].values,vocab)
lstm_test_data_loader =  DataLoader(lstm_test_data, batch_size=64, shuffle=True,collate_fn=lstm_test_data.collate)

lstm_model.eval()

true_labels = []
predicted_labels = []

# Iterate over test set
for sentences, labels in lstm_test_data_loader:
    # Forward pass
    output = lstm_model(sentences)

    # Get predicted labels
    _, predicted = torch.max(output, 1)

    # Append true and predicted labels
    true_labels.extend(labels.view(-1).tolist())
    predicted_labels.extend(predicted.tolist())

# Compute evaluation metrics
true_labels = [i-1 for i in true_labels]
print(true_labels,predicted)
accuracy = accuracy_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels,average="weighted")
f1 = f1_score(true_labels, predicted_labels,average="weighted")
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)



[3, 3, 1, 2, 1, 3, 2, 1, 0, 2, 3, 3, 0, 3, 1, 0, 0, 2, 3, 1, 2, 0, 2, 1, 3, 0, 3, 2, 0, 2, 0, 3, 1, 2, 0, 1, 2, 3, 2, 0, 3, 2, 3, 2, 2, 0, 0, 1, 3, 1, 3, 2, 2, 2, 1, 0, 0, 1, 3, 0, 3, 0, 2, 3, 3, 0, 1, 0, 3, 1, 1, 0, 3, 2, 2, 2, 2, 2, 2, 2, 0, 3, 3, 2, 3, 0, 1, 2, 2, 0, 0, 2, 3, 1, 3, 1, 1, 1, 0, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 0, 0, 3, 2, 3, 0, 3, 3, 3, 2, 2, 0, 3, 3, 1, 0, 0, 0, 1, 3, 2, 2, 0, 0, 0, 3, 2, 0, 3, 3, 0, 3, 2, 3, 1, 1, 0, 2, 0, 0, 2, 3, 2, 3, 1, 2, 0, 3, 0, 0, 0, 1, 0, 2, 2, 0, 0, 2, 1, 3, 0, 0, 3, 1, 2, 2, 2, 3, 0, 3, 3, 3, 0, 3, 2, 0, 3, 2, 2, 1, 2, 3, 0, 2, 0, 2, 3, 0, 3, 1, 1, 3, 0, 2, 0, 3, 1, 3, 0, 0, 2, 0, 0, 0, 1, 0, 3, 1, 1, 0, 2, 1, 0, 0, 3, 1, 3, 2, 2, 2, 3, 2, 2, 0, 0, 3, 0, 0, 1, 0, 3, 3, 1, 1, 0, 2, 2, 1, 3, 0, 3, 2, 2, 1, 1, 1, 3, 1, 2, 0, 2, 1, 3, 3, 1, 1, 0, 1, 3, 3, 0, 2, 2, 2, 2, 3, 1, 2, 2, 2, 2, 0, 3, 0, 2, 0, 0, 0, 1, 2, 0, 0, 2, 2, 1, 3, 0, 2, 3, 0, 2, 2, 3, 1, 2, 1, 0, 3, 0, 1, 1, 3, 2, 2, 1, 3, 2, 3, 0, 2, 0, 3, 3, 0, 1, 2, 1, 3, 2, 0, 3, 3, 2, 0, 

Final Accuracies in train - 0.78
on test 0.77 as shown as output of cells above.
The comparison with svd does not exist because I could not train the svd on RAM (memory exceeded and kernel crashed)

In [32]:
torch.save(lstm_model,"word2veclstm.pt")